# Primera aproximacipon a RL
### Entorno de control clasico de gym - CartPole-V0

In [2]:
import os
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from gym.envs.classic_control.cartpole import CartPoleEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.logger import configure

# path to save the training logs and weidths 
log_path = os.path.join("Training", "Logs", "A2C_cartPole")
A2C_path = os.path.join("Training", "Saved_Models", "A2C")

In [3]:
# se crea en entorno para el cual se va a entrenar el agente con Rl
env = CartPoleEnv() 

In [4]:
# se define un callback para obtener información de entrenamiento
eval_callback = EvalCallback(env, eval_freq=1000, verbose=0)

In [5]:
# se define el agente con un modelo (policy) de Mlp : multi-layer Perceptron 
env = DummyVecEnv([lambda: env])
model = A2C('MlpPolicy', env, seed=123, verbose=2)
#A2C?

Using cpu device


In [6]:
# se cambia la configuracion para guardar la información de entrenamiento,
# se añade un archivo csv para tener los datos disponibles.
new_logger = configure(log_path, ["csv", "tensorboard"])


2021-12-13 13:58:02.451926: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:
2021-12-13 13:58:02.452051: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
# Se ejecuta el entrenamiento del agente...
model.set_logger(new_logger)
model.learn(total_timesteps=20000, callback=eval_callback)

/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [8]:
# Se guarda el modelo...
model.save(A2C_path)

In [11]:
# Se chequa la regla aprendida por el agente
evaluate_policy(model, env, n_eval_episodes=10, render=False)

/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(1001.9, 393.2486363612721)